# Exemple d'utilisation de SimulationManager

Ce notebook démontre comment utiliser `SimulationManager` pour gérer plusieurs simulations ECL avec optimisation du partage de données.

## 1. Import des bibliothèques

In [ ]:
import sys
sys.path.append('..')

from src.factory.simulation_manager import SimulationManager
from src.core import config as cst

## 2. Cas d'usage simple - Fichiers différents

Chaque simulation utilise un fichier de données différent.

In [ ]:
# Créer le manager
manager = SimulationManager()

# Ajouter des simulations avec des fichiers différents
# Le data_identifier est automatiquement le nom du fichier
manager.add_simulation(
    simulation_name="january_analysis",
    operation_type=cst.OperationType.NON_RETAIL,
    operation_status=cst.OperationStatus.PERFORMING,
    data_file_path=r"..\sample\data\january_data.zip",
    template_file_path=r"..\sample\templates\Template_scenario1.xlsx"
)

manager.add_simulation(
    simulation_name="february_analysis",
    operation_type=cst.OperationType.NON_RETAIL,
    operation_status=cst.OperationStatus.PERFORMING,
    data_file_path=r"..\sample\data\february_data.zip",
    template_file_path=r"..\sample\templates\Template_scenario1.xlsx"
)

print("Simulations ajoutées:")
print(manager.list_simulations())

## 3. Cas d'usage avec partage de données

Plusieurs simulations partagent le même fichier de données mais avec des templates différents.

In [ ]:
# Créer un nouveau manager
manager_shared = SimulationManager()

# Même fichier de données, templates différents
data_path = r"..\sample\data\sample_non_retail.zip"

manager_shared.add_simulation(
    simulation_name="scenario_base",
    operation_type=cst.OperationType.NON_RETAIL,
    operation_status=cst.OperationStatus.PERFORMING,
    data_file_path=data_path,
    template_file_path=r"..\sample\templates\Template_scenario1.xlsx"
)

manager_shared.add_simulation(
    simulation_name="scenario_stress",
    operation_type=cst.OperationType.NON_RETAIL,
    operation_status=cst.OperationStatus.PERFORMING,
    data_file_path=data_path,  # Même fichier
    template_file_path=r"..\sample\templates\Template_scenario2.xlsx"
)

manager_shared.add_simulation(
    simulation_name="scenario_optimistic",
    operation_type=cst.OperationType.NON_RETAIL,
    operation_status=cst.OperationStatus.PERFORMING,
    data_file_path=data_path,  # Même fichier
    template_file_path=r"..\sample\templates\Template_scenario3.xlsx"
)

print("\n📊 Résumé du partage de données:")
summary = manager_shared.get_sharing_summary()
for data_file, simulations in summary.items():
    print(f"  {data_file}: {len(simulations)} simulations")
    for sim in simulations:
        print(f"    - {sim}")

## 4. Cas d'usage avec identifiants explicites

Utiliser des identifiants personnalisés pour grouper les données (utile pour des copies ou des sources différentes).

In [ ]:
# Créer un nouveau manager
manager_custom = SimulationManager()

# Simuler des fichiers uploadés avec le même nom logique
# (par exemple, des copies temporaires du même fichier)
temp_path_1 = r"..\temp\upload_abc123.zip"
temp_path_2 = r"..\temp\upload_xyz789.zip"

# Les deux simulations utilisent le même identifiant → partage de données
manager_custom.add_simulation(
    simulation_name="analysis_v1",
    operation_type=cst.OperationType.NON_RETAIL,
    operation_status=cst.OperationStatus.PERFORMING,
    data_file_path=temp_path_1,
    template_file_path=r"..\sample\templates\Template_scenario1.xlsx",
    data_identifier="quarterly_data_Q1.zip"  # Identifiant explicite
)

manager_custom.add_simulation(
    simulation_name="analysis_v2",
    operation_type=cst.OperationType.NON_RETAIL,
    operation_status=cst.OperationStatus.PERFORMING,
    data_file_path=temp_path_2,  # Chemin différent
    template_file_path=r"..\sample\templates\Template_scenario2.xlsx",
    data_identifier="quarterly_data_Q1.zip"  # Même identifiant → partage!
)

print("\n📊 Partage avec identifiants personnalisés:")
for data_id, sims in manager_custom.get_sharing_summary().items():
    if len(sims) > 1:
        print(f"  ✅ '{data_id}' partagé par: {', '.join(sims)}")

## 5. Préparation des simulations

Les données sont chargées UNE SEULE FOIS par groupe de partage.

In [ ]:
# Préparer toutes les simulations
print("\n🚀 Préparation des simulations...")
manager_shared.prepare_all_simulations()

print("\n✅ Simulations prêtes!")

## 6. Accès aux simulations préparées

In [ ]:
# Récupérer une simulation spécifique
factory_base = manager_shared.get_simulation("scenario_base")
factory_stress = manager_shared.get_simulation("scenario_stress")

print("\nInformations sur les simulations:")
print(f"  scenario_base: {factory_base.ecl_operation_data.df.shape}")
print(f"  scenario_stress: {factory_stress.ecl_operation_data.df.shape}")

# Vérifier que les DataFrames sont bien des copies indépendantes
print(f"\n  Les DataFrames sont des copies indépendantes: {factory_base.ecl_operation_data.df is not factory_stress.ecl_operation_data.df}")

## 7. Continuer avec les calculs ECL

In [ ]:
# Exemple: calculer les time steps pour une simulation
# factory_base.get_time_steps()
# factory_base.get_scenarios()
# etc.

print("\n💡 Les simulations sont prêtes pour les calculs ECL!")

## Résumé des avantages

✅ **Optimisation automatique**: Les données identiques ne sont chargées qu'une fois  
✅ **Flexible**: Fonctionne avec ou sans identifiants explicites  
✅ **Simple**: API intuitive pour notebooks et scripts  
✅ **Compatible Streamlit**: Utilise `uploaded_file.name` comme identifiant  
✅ **Sécurisé**: Chaque simulation reçoit une copie indépendante des données